In [1]:
import uxarray as ux
import xarray as xr
import numpy as np

# isolated function from mpas_tools
from mpas_source import scrip_from_mpas

# MPAS Mesh (All Variables Included)

In [77]:
input_path = "data/x1.40962.static.nc"
output_path = "data/x1.40962.static.fixed.nc"

In [78]:
mpas_ds = xr.open_dataset(input_path)
mpas_ds

<xarray.Dataset>
Dimensions:                   (Time: 1, nCells: 40962, nEdges: 122880,
                               nVertices: 81920, TWO: 2, maxEdges: 10,
                               maxEdges2: 20, vertexDegree: 3, R3: 3,
                               nMonths: 12, FIFTEEN: 15, TWENTYONE: 21)
Dimensions without coordinates: Time, nCells, nEdges, nVertices, TWO, maxEdges,
                                maxEdges2, vertexDegree, R3, nMonths, FIFTEEN,
                                TWENTYONE
Data variables: (12/73)
    xtime                     (Time) |S64 ...
    latCell                   (nCells) float32 ...
    lonCell                   (nCells) float32 ...
    xCell                     (nCells) float32 ...
    yCell                     (nCells) float32 ...
    zCell                     (nCells) float32 ...
    ...                        ...
    ol4                       (nCells) float32 ...
    deriv_two                 (nEdges, TWO, FIFTEEN) float32 ...
    advCells                  (nCells, TWENTYONE) int32 ...
    defc_a                    (nCells, maxEdges) float32 ...
    defc_b                    (nCells, maxEdges) float32 ...
    coeffs_reconstruct        (nCells, maxEdges, R3) float32 ...
Attributes: (12/59)
    model_name:                       mpas
    core_name:                        init_atmosphere
    source:                           MPAS
    Conventions:                      MPAS
    git_version:                      v7.0
    on_a_sphere:                      YES
    ...                               ...
    config_pio_stride:                1
    config_block_decomp_file_prefix:  x1.40962.graph.info.part.
    config_number_of_blocks:          0
    config_explicit_proc_decomp:      NO
    config_proc_decomp_file_prefix:   graph.info.part.
    file_id:                          937lresxyt

In [79]:
print(f"lonVertex Min: {mpas_ds['lonVertex'].values.min()}")
print(f"lonVertex Max: {mpas_ds['lonVertex'].values.max()}")   

lonVertex Min: -3.14156436920166
lonVertex Max: 3.141563653945923


In [80]:
# lonVertex must be in range [0, 2pi]
mpas_ds['lonVertex'] += np.pi 
mpas_ds['lonVertex'] = mpas_ds['lonVertex'].clip(0.0, 2.0*np.pi)

In [81]:
print(f"lonVertex Min: {mpas_ds['lonVertex'].values.min()}")
print(f"lonVertex Max: {mpas_ds['lonVertex'].values.max()}")   

lonVertex Min: 2.8371810913085938e-05
lonVertex Max: 6.283156394958496


In [82]:
# save to a new netCDF file with corrected range 
mpas_ds.to_netcdf(output_path)

In [40]:
mpas_file = "data/x1.40962.static.fixed.nc"
scrip_file = "data/x1.40962.static.scrip.nc"

In [83]:
# reads MPAS netCDF, outputs SCRIP netCDF
scrip_from_mpas(mpas_file, scrip_file, useLandIceMask=False)

 -- Landice Masks are disabled

Input latCell min/max values (radians): -1.5707963267948966, 1.5707963267948966
Input lonCell min/max values (radians): 0.0, 6.281987301405775
Calculated grid_center_lat min/max values (radians): -1.5707963267948966, 1.5707963267948966
Calculated grid_center_lon min/max values (radians): 0.0, 6.281987301405775
Calculated grid_area min/max values (sq radians): 2.7252165100350787e-06, 4.985611904310155e-06
Creation of SCRIP file is complete.


In [84]:
scrip_ds = xr.open_dataset(scrip_file)
scrip_ds

<xarray.Dataset>
Dimensions:          (grid_size: 2621442, grid_corners: 10, grid_rank: 1)
Dimensions without coordinates: grid_size, grid_corners, grid_rank
Data variables:
    grid_center_lat  (grid_size) float64 ...
    grid_center_lon  (grid_size) float64 ...
    grid_corner_lat  (grid_size, grid_corners) float64 ...
    grid_corner_lon  (grid_size, grid_corners) float64 ...
    grid_imask       (grid_size) int32 ...
    grid_dims        (grid_rank) int32 ...
    grid_area        (grid_size) float64 ...

In [85]:
%%time
# [mpas] -> [scrip] -> [ugrid]
ugrid_mpas = ux.Grid(scrip_ds)
ugrid_mpas.ds

CPU times: user 27.8 s, sys: 1.49 s, total: 29.3 s
Wall time: 31.6 s


<xarray.Dataset>
Dimensions:           (nMesh2_node: 5242880, grid_size: 2621442,
                       nMesh2_face: 2621442, nMaxMesh2_face_nodes: 10)
Dimensions without coordinates: nMesh2_node, grid_size, nMesh2_face,
                                nMaxMesh2_face_nodes
Data variables:
    Mesh2_node_x      (nMesh2_node) float64 0.0 4.441e-16 ... 6.283 6.283
    Mesh2_node_y      (nMesh2_node) float64 0.6205 -1.089 ... 0.9338 1.298
    Mesh2_face_x      (grid_size) float64 ...
    Mesh2_face_y      (grid_size) float64 ...
    Mesh2_face_nodes  (nMesh2_face, nMaxMesh2_face_nodes) int64 3931384 ... 2...
    Mesh2             float64 nan

# MPAS Mesh (UXarray Issue #175, Missing AreaCell)
https://github.com/UXARRAY/uxarray/issues/175

In [44]:
input_path = "data/x1.655362.grid_subset.nc"
output_path = "data/x1.655362.grid_subset.fixed.nc"

In [36]:
mpas_ds = xr.open_dataset(input_path)
mpas_ds

<xarray.Dataset>
Dimensions:         (nCells: 655362, nVertices: 1310720, vertexDegree: 3,
                     maxEdges: 10)
Dimensions without coordinates: nCells, nVertices, vertexDegree, maxEdges
Data variables:
    lonCell         (nCells) float32 ...
    latCell         (nCells) float32 ...
    lonVertex       (nVertices) float32 ...
    latVertex       (nVertices) float32 ...
    cellsOnVertex   (nVertices, vertexDegree) int32 ...
    verticesOnCell  (nCells, maxEdges) int32 ...
    nEdgesOnCell    (nCells) int32 ...
Attributes: (12/54)
    model_name:                       mpas
    core_name:                        init_atmosphere
    source:                           MPAS
    Conventions:                      MPAS
    git_version:                      v6.0-dirty
    on_a_sphere:                      YES
    ...                               ...
    config_block_decomp_file_prefix:  x1.655362.graph.info.part.
    config_number_of_blocks:          0
    config_explicit_proc_decomp:      NO
    config_proc_decomp_file_prefix:   graph.info.part.
    file_id:                          ytj0fh0qmd
    NCO:                              netCDF Operators version 4.9.5 (Homepag...

In [37]:
# lonVertex must be in range [0, 2pi]
mpas_ds['lonVertex'] += np.pi 
mpas_ds['lonVertex'] = mpas_ds['lonVertex'].clip(0.0, 2.0*np.pi)

In [38]:
# save to a new netCDF file with corrected range 
mpas_ds.to_netcdf(output_path)

In [39]:
mpas_file = "data/x1.655362.grid_subset.fixed.nc"
scrip_file = "data/x1.655362.grid_subset.scrip.nc"

In [40]:
# reads MPAS netCDF, outputs SCRIP netCDF
scrip_from_mpas(mpas_file, scrip_file, useLandIceMask=False)

 -- Landice Masks are disabled



KeyError: 'areaCell'

# MPAS Mesh (UXarray Issue #175, New Dataset)
https://github.com/UXARRAY/uxarray/issues/175

In [4]:
input_path = "data/x1.2621442.grid.nc"
output_path = "data/x1.2621442.grid.fix.nc"

In [5]:
mpas_ds = xr.open_dataset(input_path)
mpas_ds

<xarray.Dataset>
Dimensions:            (nCells: 2621442, nEdges: 7864320, nVertices: 5242880,
                        TWO: 2, maxEdges: 10, maxEdges2: 20, vertexDegree: 3)
Dimensions without coordinates: nCells, nEdges, nVertices, TWO, maxEdges,
                                maxEdges2, vertexDegree
Data variables: (12/38)
    latCell            (nCells) float64 ...
    lonCell            (nCells) float64 ...
    meshDensity        (nCells) float64 ...
    xCell              (nCells) float64 ...
    yCell              (nCells) float64 ...
    zCell              (nCells) float64 ...
    ...                 ...
    cellsOnCell        (nCells, maxEdges) int32 ...
    verticesOnCell     (nCells, maxEdges) int32 ...
    verticesOnEdge     (nEdges, TWO) int32 ...
    edgesOnVertex      (nVertices, vertexDegree) int32 ...
    cellsOnVertex      (nVertices, vertexDegree) int32 ...
    kiteAreasOnVertex  (nVertices, vertexDegree) float64 ...
Attributes:
    on_a_sphere:        YES             
    sphere_radius:      1.0
    np:                 2621442
    n_scvt_iterations:  100
    eps:                1e-10
    Convergence:        L2

In [7]:
mpas_ds['lonVertex'].min(), mpas_ds['lonVertex'].max()

(<xarray.DataArray 'lonVertex' ()>
 array(-3.14159265),
 <xarray.DataArray 'lonVertex' ()>
 array(3.14159265))

In [8]:
# lonVertex must be in range [0, 2pi]
mpas_ds['lonVertex'] += np.pi 
mpas_ds['lonVertex'] = mpas_ds['lonVertex'].clip(0.0, 2.0*np.pi)

# save to a new netCDF file with corrected range 
mpas_ds.to_netcdf(output_path)

In [9]:
mpas_file = "data/x1.2621442.grid.fix.nc"
scrip_file = "data/x1.2621442.grid.scrip.nc"

In [10]:
# reads MPAS netCDF, outputs SCRIP netCDF
scrip_from_mpas(mpas_file, scrip_file, useLandIceMask=False)

 -- Landice Masks are disabled

Input latCell min/max values (radians): -1.5707963267948966, 1.5707963267948966
Input lonCell min/max values (radians): 0.0, 6.281987301405775
Calculated grid_center_lat min/max values (radians): -1.5707963267948966, 1.5707963267948966
Calculated grid_center_lon min/max values (radians): 0.0, 6.281987301405775
Calculated grid_area min/max values (sq radians): 2.7252165100350787e-06, 4.985611904310155e-06
Creation of SCRIP file is complete.


In [11]:
scrip_ds = xr.open_dataset(scrip_file)
scrip_ds

<xarray.Dataset>
Dimensions:          (grid_size: 2621442, grid_corners: 10, grid_rank: 1)
Dimensions without coordinates: grid_size, grid_corners, grid_rank
Data variables:
    grid_center_lat  (grid_size) float64 ...
    grid_center_lon  (grid_size) float64 ...
    grid_corner_lat  (grid_size, grid_corners) float64 ...
    grid_corner_lon  (grid_size, grid_corners) float64 ...
    grid_imask       (grid_size) int32 ...
    grid_dims        (grid_rank) int32 ...
    grid_area        (grid_size) float64 ...

In [12]:
%%time
# [mpas] -> [scrip] -> [ugrid]
ugrid_mpas = ux.Grid(scrip_ds)
ugrid_mpas.ds

CPU times: user 27.7 s, sys: 738 ms, total: 28.4 s
Wall time: 28.9 s


<xarray.Dataset>
Dimensions:           (nMesh2_node: 5242880, grid_size: 2621442,
                       nMesh2_face: 2621442, nMaxMesh2_face_nodes: 10)
Dimensions without coordinates: nMesh2_node, grid_size, nMesh2_face,
                                nMaxMesh2_face_nodes
Data variables:
    Mesh2_node_x      (nMesh2_node) float64 0.0 4.441e-16 ... 6.283 6.283
    Mesh2_node_y      (nMesh2_node) float64 0.6205 -1.089 ... 0.9338 1.298
    Mesh2_face_x      (grid_size) float64 ...
    Mesh2_face_y      (grid_size) float64 ...
    Mesh2_face_nodes  (nMesh2_face, nMaxMesh2_face_nodes) int64 3931384 ... 2...
    Mesh2             float64 nan

# Notes

* Converting from $[MPAS] \rightarrow [SCRIP] \rightarrow [UGRID]$ allows us to read in MPAS grids with UXarray
* `mpas_tools.scrip_from_mpas()` is written to read in an MPAS file and output a SCRIP file to disk which is not idea for UXarray
* Rewriting this function to input and output a `Xarray.Dataset` instead would plug in with our existing implementation

## Required MPAS Variables
* $latCell, lonCell$
* $latVertex, lonVertex$
* $verticiesOnCell$
* $nEdgesOnCell$
* $areaCell$

